In [1]:
import pandas as pd
import geojson

In [2]:
from geojson import Feature, Point, FeatureCollection

houses_df = pd.read_csv('../data/curated/property/property_4_lat_lon.csv')  
cbd = pd.read_csv('../data/raw/external/melbourne_cbd_coordinates.csv')  
cbd_df = cbd.rename(columns={'Longitude': 'longitude', 'Latitude': 'latitude'})

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
cbd_df = cbd_df.dropna(subset=['longitude', 'latitude'])


### convert geojson
def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'),  
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

### apply the function
house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

cbd_features = cbd_df.apply(lambda row: create_geojson_feature(row, 'cbd'), axis=1).tolist()
cbd_feature_collection = FeatureCollection(cbd_features)


### save geojson
houses_geojson_output_path = '../data/raw/external/API/4/property_4_geojson.geojson'
cbd_geojson_output_path = '../data/raw/external/API/4/melbourne_cbd_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(cbd_geojson_output_path, 'w') as f:
    geojson.dump(cbd_feature_collection, f)


print("geojson created")


geojson created


In [3]:
import geojson
import openrouteservice
import pandas as pd
import time

api_key = '5b3ce3597851110001cf6248c2b2602de2dc43afa9f186cdfc39caca' 
client = openrouteservice.Client(key=api_key)

### get geo files
with open('../data/raw/external/API/4/property_4_geojson.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data/raw/external/API/4/melbourne_cbd_geojson.geojson', 'r') as f:
    cbd_geojson = geojson.load(f)

### function of get the latitude and longitude
def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
cbd_coordinates = extract_coordinates(cbd_geojson)

### apply the function
house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
cbd_coords = [(lon, lat) for lat, lon, _ in cbd_coordinates]


### the function of calculate the distance
def calculate_distance_matrix_in_batches(house_coords, cbd_coords, batch_size=100):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + cbd_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(cbd_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            ### find the mini distance
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
              
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_cbd_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_cbd = cbd_coordinates[closest_cbd_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_cbd': closest_cbd,
                        'minimum_distance_cbd': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)  
    
    return all_shortest_distances

### apply the distance function
shortest_distances = calculate_distance_matrix_in_batches(house_coords, cbd_coords, batch_size=100)

### save
shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data/curated/external/API/4/closest_cbd_distance_final_4.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"saved {output_path}")


saved ../data/curated/external/API/4/closest_cbd_distance_final_4.csv
